In [1]:
import pywren_ibm_cloud as pywren

In [2]:
config = {'ibm_cf':  {'endpoint': 'https://eu-gb.functions.cloud.ibm.com', 
                      'namespace': 'cloudbutton@uvigo.es_dev', 
                      'api_key': 'c8a9e3ec-51c9-413b-ac23-e10c3ccb71e1:k3GoGB7GRgyNFYI3ob97GAuKt8ORPYJ9eWjCfIygD0d2xeR9aowjaQlvgm7HhlPm'}, 
          'ibm_cos': {'endpoint': 'https://s3.eu.cloud-object-storage.appdomain.cloud', 
                      'api_key' : 'RRJtoAasvR3PJtezkSNWL5HLowZ5D3s9FDINBlxIBosV'},
           'pywren' : {'storage_bucket' : 'cloudbuttonbucket01'}}
#https://eu-gb.functions.cloud.ibm.com/api/v1/namespaces/cloudbutton%40uvigo.es_dev/actions/testing/
# https://eu-gb.functions.cloud.ibm.com

In [3]:
pw = pywren.ibm_cf_executor(config=config)

PyWren v1.3.0 init for IBM Cloud Functions - Namespace: cloudbutton@uvigo.es_dev - Region: eu_gb


In [9]:
def my_function(x):
    return x + 2

In [10]:
pw.map(my_function, [2, 3, 4, 2, 4, 5], runtime_memory=128)

ExecutorID 9f3ae8/0 | JobID M002 - Selected Runtime: ibmfunctions/action-python-v3.6 - 128MB 
ExecutorID 9f3ae8/0 | JobID M002 - Uploading function and data - Total: 591.0B
ExecutorID 9f3ae8/0 | JobID M002 - Starting function invocation: my_function()  - Total: 6 activations


In [11]:
pw.get_result()

ExecutorID 9f3ae8/0 - Getting results...


[4, 5, 6, 4, 6, 7]

In [12]:
pw.clean()

ExecutorID 9f3ae8/0 - Cleaning temporary data


In [48]:
pw.map(my_function, range(10), runtime_memory=128)

ExecutorID 7e5ce9/17 | JobID M002 - Selected Runtime: ibmfunctions/action-python-v3.6 - 128MB 
ExecutorID 7e5ce9/17 | JobID M002 - Uploading function and data - Total: 676.0B
ExecutorID 7e5ce9/17 | JobID M002 - Starting function invocation: my_function()  - Total: 10 activations


In [49]:
print(pw.get_result())
pw.clean()

ExecutorID 7e5ce9/17 - Getting results...



[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
ExecutorID 7e5ce9/17 - Cleaning temporary data


In [27]:
from time import time
from random import random

In [28]:
MAP_INSTANCES = 10


class EstimatePI:
    randomize_per_map = 10000000

    def __init__(self):
        self.total_randomize_points = MAP_INSTANCES * self.randomize_per_map

    def __str__(self):
        return "Total Randomize Points: {:,}".format(self.randomize_per_map * MAP_INSTANCES)

    @staticmethod
    def predicate():
        x = random()
        y = random()
        return (x ** 2) + (y ** 2) <= 1

    def randomize_points(self, data):
        in_circle = 0
        for _ in range(self.randomize_per_map):
            in_circle += self.predicate()
        return float(in_circle / self.randomize_per_map)

    def process_in_circle_points(self, results):
        in_circle_percent = 0
        for map_result in results:
            in_circle_percent += map_result
        estimate_PI = float(4 * (in_circle_percent / MAP_INSTANCES))
        return estimate_PI

In [35]:
iterdata = [0] * MAP_INSTANCES
est_pi = EstimatePI()

start_time = time()
print("Monte Carlo simulation for estimating PI spawing over {} IBM Cloud Function invocations".format(MAP_INSTANCES))
# obtain PyWren-IBM-Cloud executor
pw = pywren.ibm_cf_executor(config=config, runtime_memory=128)
# execute the code
pw.map_reduce(est_pi.randomize_points, iterdata, est_pi.process_in_circle_points)
#get results
result = pw.get_result()
elapsed = time()
print(str(est_pi))
print("Estimation of Pi: ", result)
print("\nCompleted in: " + str(elapsed - start_time) + " seconds")

Monte Carlo simulation for estimating PI spawing over 10 IBM Cloud Function invocations
PyWren v1.3.0 init for IBM Cloud Functions - Namespace: cloudbutton@uvigo.es_dev - Region: eu_gb
ExecutorID 7e5ce9/15 | JobID M000 - Selected Runtime: ibmfunctions/action-python-v3.6 - 128MB 
ExecutorID 7e5ce9/15 | JobID M000 - Uploading function and data - Total: 5.7KiB
ExecutorID 7e5ce9/15 | JobID M000 - Starting function invocation: randomize_points()  - Total: 10 activations
ExecutorID 7e5ce9/15 | JobID R000 - Selected Runtime: ibmfunctions/action-python-v3.6 - 128MB 
ExecutorID 7e5ce9/15 | JobID R000 - Uploading function and data - Total: 9.9KiB
ExecutorID 7e5ce9/15 | JobID R000 - Starting function invocation: process_in_circle_points()  - Total: 1 activations
ExecutorID 7e5ce9/15 - Getting results...



Total Randomize Points: 100,000,000
Estimation of Pi:  3.1420856000000006

Completed in: 28.32344627380371 seconds


In [36]:
pw.clean()

ExecutorID 7e5ce9/15 - Cleaning temporary data


In [37]:
def my_map_function(id, x):
    print("I'm activation number {}".format(id))
    return x + 10


In [38]:
iterdata = [1, 2, 3, 4]
pw = pywren.ibm_cf_executor(config=config)
pw.map(my_map_function, iterdata)
print(pw.get_result())


PyWren v1.3.0 init for IBM Cloud Functions - Namespace: cloudbutton@uvigo.es_dev - Region: eu_gb
ExecutorID 7e5ce9/16 | JobID M000 - Selected Runtime: ibmfunctions/action-python-v3.6 - 128MB 
ExecutorID 7e5ce9/16 | JobID M000 - Uploading function and data - Total: 620.0B
ExecutorID 7e5ce9/16 | JobID M000 - Starting function invocation: my_map_function()  - Total: 4 activations
ExecutorID 7e5ce9/16 - Getting results...



[11, 12, 13, 14]


In [82]:
pw.clean()

ExecutorID f32446/28 - Cleaning temporary data


In [85]:
params = [[1, 2, 3, 4, 5], 5]

def sum_list_mult(list_of_numbers, x):
    total = 0
    for num in list_of_numbers:
        total = total+num
    return total*x
start_time = time()
pw = pywren.ibm_cf_executor(config=config)
pw.call_async(sum_list_mult, params)
print (pw.get_result())
elapsed = time()
print("\nCompleted in: " + str(elapsed - start_time) + " seconds")

PyWren v1.3.0 init for IBM Cloud Functions - Namespace: cloudbutton@uvigo.es_dev - Region: eu_gb
ExecutorID f32446/30 | JobID A000 - Selected Runtime: ibmfunctions/action-python-v3.6 - 256MB 
ExecutorID f32446/30 | JobID A000 - Uploading function and data - Total: 587.0B
ExecutorID f32446/30 | JobID A000 - Starting function invocation: sum_list_mult()  - Total: 1 activations
ExecutorID f32446/30 - Getting results...



75

Completed in: 2.0621018409729004 seconds


In [104]:
iterdata = [  # Init list of parameters for PyWren
           [1, 2],  # Parameters for function1
           [3, 4],  # Parameters for function2
           [5, 6],  # Parameters for function3
           ]  # End list of parameters for PyWren

def my_function(x, y):
    return x + y

pw = pywren.ibm_cf_executor(config=config)
pw.map(my_function, iterdata)
print (pw.get_result())

PyWren v1.3.0 init for IBM Cloud Functions - Namespace: cloudbutton@uvigo.es_dev - Region: eu_gb
ExecutorID f32446/46 | JobID M000 - Selected Runtime: ibmfunctions/action-python-v3.6 - 256MB 
ExecutorID f32446/46 | JobID M000 - Uploading function and data - Total: 553.0B
ExecutorID f32446/46 | JobID M000 - Starting function invocation: my_function()  - Total: 3 activations
ExecutorID f32446/46 - Getting results...



[3, 7, 11]


In [87]:
iterdata = [  # Init list of parameters for PyWren
           {'x': 1, 'y': 2},  # Parameters for function1
           {'x': 3, 'y': 4},  # Parameters for function2
           {'x': 5, 'y': 6},  # Parameters for function3
           ]  # End list of parameters for PyWren

def my_function(x, y):
    return x + y

pw = pywren.ibm_cf_executor(config=config)
pw.map(my_function, iterdata)
print (pw.get_result())

PyWren v1.3.0 init for IBM Cloud Functions - Namespace: cloudbutton@uvigo.es_dev - Region: eu_gb
ExecutorID f32446/32 | JobID M000 - Selected Runtime: ibmfunctions/action-python-v3.6 - 256MB 
ExecutorID f32446/32 | JobID M000 - Uploading function and data - Total: 552.0B
ExecutorID f32446/32 | JobID M000 - Starting function invocation: my_function()  - Total: 3 activations
ExecutorID f32446/32 - Getting results...



[3, 7, 11]


In [88]:
iterdata = [  # Init list of parameters for PyWren
           [[1, 2]],  # Parameters for function1
           [[3, 4]],  # Parameters for function2
           [[5, 6]],  # Parameters for function3
           ]  # End list of parameters for PyWren

def sum_list(list_of_numbers):
    total = 0
    for num in list_of_numbers:
        total = total+num
    return total

pw = pywren.ibm_cf_executor(config=config)
pw.map(sum_list, iterdata)
print (pw.get_result())

PyWren v1.3.0 init for IBM Cloud Functions - Namespace: cloudbutton@uvigo.es_dev - Region: eu_gb
ExecutorID f32446/33 | JobID M000 - Selected Runtime: ibmfunctions/action-python-v3.6 - 256MB 
ExecutorID f32446/33 | JobID M000 - Uploading function and data - Total: 642.0B
ExecutorID f32446/33 | JobID M000 - Starting function invocation: sum_list()  - Total: 3 activations
ExecutorID f32446/33 - Getting results...



[3, 7, 11]


In [102]:
iterdata = [  # Init list of parameters for PyWren
           [[1, 2, 3, 4, 5], 2],  # Parameters for function1
           [[6, 7, 8, 9, 10], 3],  # Parameters for function2
           [[11, 12, 13, 14, 15], 4],  # Parameters for function3
           ]  # End list of parameters for PyWren

def sum_list_mult(list_of_numbers, x):
    total = 0
    for num in list_of_numbers:
        total = total+num
    return total*x

pw = pywren.ibm_cf_executor(config=config)
pw.map(sum_list_mult, iterdata)
print (pw.get_result())

PyWren v1.3.0 init for IBM Cloud Functions - Namespace: cloudbutton@uvigo.es_dev - Region: eu_gb
ExecutorID f32446/44 | JobID M000 - Selected Runtime: ibmfunctions/action-python-v3.6 - 256MB 
ExecutorID f32446/44 | JobID M000 - Uploading function and data - Total: 696.0B
ExecutorID f32446/44 | JobID M000 - Starting function invocation: sum_list_mult()  - Total: 3 activations
ExecutorID f32446/44 - Getting results...



[30, 120, 260]


In [50]:
import gdal

ModuleNotFoundError: No module named 'gdal'

In [51]:
import pdal

ModuleNotFoundError: No module named 'pdal'